# LAB | Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

1. A **vector index (Pinecone)** to store and run semantic search

    (This is a database that stores embeddings (vector representations) of text. It lets you do **semantic search**, meaning it finds text that means similar things—not just text that looks similar.)
2. A **retriever model** for embedding context passages
    
    (This model turns each context paragraph into a vector (embedding).)
3. A **reader model** to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

Let's get started by installing the packages needed for notebook to run:

In [27]:
from google.colab import userdata
import os

# Retrieve the API keys
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

# Set as environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

print("OpenAI API key loaded and set as environment variable.")
print("Pinecone API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.
Pinecone API key loaded and set as environment variable.


# Install Dependencies

In [2]:
#!pip install -qU datasets pinecone-client sentence-transformers torch

# Load Dataset

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [3]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
print(df.shape)
df.head()

(87599, 5)


,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [5]:
# select only title and context column
df = df[["title", "context"]]

# drop rows containing duplicate context passages
df = df.drop_duplicates(subset=["context"]).reset_index(drop=True)
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,The university is the major seat of the Congre...
3,University_of_Notre_Dame,The College of Engineering was established in ...
4,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
18886,Kathmandu,"Institute of Medicine, the central college of ..."
18887,Kathmandu,Football and Cricket are the most popular spor...
18888,Kathmandu,The total length of roads in Nepal is recorded...
18889,Kathmandu,The main international airport serving Kathman...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [8]:
#!pip install -qU langchain-pinecone pinecone-notebooks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2

In [9]:

from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)

Now we create a new Pinecone index called `"extractive-question-answering"` — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

1. `metric = "cosine"`

    This means Pinecone will measure how similar two vectors are using ***cosine similarity***, which is good for text embeddings.
2. `dimension = 384`

    This means every embedding you store will be a vector with 384 numbers.

    ***Why?*** : Because the retriever model you’re using produces vectors of size 384, so the index must match that size.

In [10]:
index_name = "extractive-question-answering"

# check if the extractive-question-answering index exists
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=384, # As mentioned in text cell '58028e12'
        metric="cosine", # As mentioned in text cell '58028e12'
        spec=spec
    )
# connect to extractive-question-answering index we created
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [12]:
import torch
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)

print("Retriever loaded successfully on", device)


retriever

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retriever loaded successfully on cuda


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

---
---

🟦 You have a Pinecone index : Think: a smart vector-search database.

🟩 You upsert vectors into it: Think: store vectors inside it.

🟥 You query it: Think: search for similar vectors inside it.

In [13]:
from tqdm.auto import tqdm

batch_size = 64 # we will use batches of 64

for i in tqdm(range(0, len(df), batch_size)): # tqdm() just gives a progress bar so you can see progress.
    # find end of batch
    i_end = min(i + batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end] # .iloc = “integer-location based indexing” → you select rows by their integer position, not by label.
    # generate embeddings for batch
    emb = retriever.encode(batch["context"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{x}" for x in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891,
 'vector_type': 'dense'}

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [15]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(
    tokenizer=model_name,
              model=model_name,
              task='question-answering',
              device=device)
reader

Device set to use cuda


Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [16]:
# gets context passages from the pinecone index
def get_context(question, top_k=3):
    # convert question to vector
    xq = retriever.encode([question]).tolist()

    # search Pinecone using keyword arguments
    results = index.query(
        vector=xq[0],   # xq is a list of one vector
        top_k=top_k,
        include_metadata=True
    )

    # extract the text passages
    contexts = [match['metadata']['context'] for match in results['matches']]

    return contexts


In [17]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [18]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 1)
context

['Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of natural gas (in 2013), which makes Egypt as the largest oil producer not member of the Organization of the Petroleum Exporting Countries (OPEC) and the second-largest dry natural gas producer in Africa. In 2013, Egypt was the largest consumer of oil and natural gas in Africa, as more than 20% of total oil consumption and more than 40% of total dry natural gas consumption in Africa. Also, Egypt possesses the largest oil refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is currently planning to build its first nuclear power plant in El Dabaa city, northern Egypt.']

As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [19]:
extract_answer(question, context)

[{'answer': '691,000 bbl/d',
  'context': 'Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of '
             'natural gas (in 2013), which makes Egypt as the largest oil '
             'producer not member of the Organization of the Petroleum '
             'Exporting Countries (OPEC) and the second-largest dry natural '
             'gas producer in Africa. In 2013, Egypt was the largest consumer '
             'of oil and natural gas in Africa, as more than 20% of total oil '
             'consumption and more than 40% of total dry natural gas '
             'consumption in Africa. Also, Egypt possesses the largest oil '
             'refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is '
             'currently planning to build its first nuclear power plant in El '
             'Dabaa city, northern Egypt.',
  'end': 33,
  'score': 0.9999855201993455,
  'start': 20}]


The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [20]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, '
             'Hurley and Chen developed the idea for YouTube during the early '
             'months of 2005, after they had experienced difficulty sharing '
             "videos that had been shot at a dinner party at Chen's apartment "
             'in San Francisco. Karim did not attend the party and denied that '
             'it had occurred, but Chen commented that the idea that YouTube '
             'was founded after a dinner party "was probably very strengthened '
             'by marketing ideas around creating a story that was very '
             'digestible".',
  'end': 79,
  'score': 0.9999276399612427,
  'start': 64}]


In [21]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity '
             'and general relativity. He also made important contributions to '
             'statistical mechanics, especially his mathematical treatment of '
             'Brownian motion, his resolution of the paradox of specific '
             'heats, and his connection of fluctuations and dissipation. '
             'Despite his reservations about its interpretation, Einstein also '
             'made contributions to quantum mechanics and, indirectly, quantum '
             'field theory, primarily through his theoretical studies of the '
             'photon.',
  'end': 86,
  'score': 0.9500370025634766,
  'start': 29}]


Let's run another question. This time for top 3 context passages from the retriever.

In [22]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

The result looks pretty good.

In [23]:
pc.delete_index(index_name)

### Add a few more questions. What did you observe?

How to fix it:

In [41]:
import os

# Check keys
print("OpenAI Key:", bool(os.environ.get("OPENAI_API_KEY")))
print("Pinecone Key:", bool(os.environ.get("PINECONE_API_KEY")))

OpenAI Key: True
Pinecone Key: True


In [40]:
from pinecone import Pinecone, ServerlessSpec
from datasets import load_dataset
import os

df = load_dataset("squad", split="train").to_pandas()
df = df[["title", "context"]]
df = df.drop_duplicates(subset=["context"]).reset_index(drop=True)
df

# Setup and connect to Pinecode
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'
)


In [34]:
#print(df.shape)
#df.head()
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,The university is the major seat of the Congre...
3,University_of_Notre_Dame,The College of Engineering was established in ...
4,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
18886,Kathmandu,"Institute of Medicine, the central college of ..."
18887,Kathmandu,Football and Cricket are the most popular spor...
18888,Kathmandu,The total length of roads in Nepal is recorded...
18889,Kathmandu,The main international airport serving Kathman...


### Initialise Pinecone index

In [36]:
index_name = "extractive-question-answering"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=spec
    )

index = pc.Index(index_name)

### Initialise Retriever

In [37]:
import torch
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)

print("Retriever loaded successfully on", device)


retriever

Retriever loaded successfully on cuda


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Initialise Reader

In [38]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'

reader = pipeline(
    tokenizer=model_name,
              model=model_name,
              task='question-answering',
              device=device)
reader

Device set to use cuda


### Generate Embedding Upsert

In [39]:
from tqdm.auto import tqdm

batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):

    i_end = min(i + batch_size, len(df))

    batch = df.iloc[i:i_end]

    emb = retriever.encode(batch["context"].tolist()).tolist()

    meta = batch.to_dict(orient="records")

    ids = [f"{x}" for x in range(i, i_end)]

    to_upsert = list(zip(ids, emb, meta))

    _ = index.upsert(vectors=to_upsert)

index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891,
 'vector_type': 'dense'}

In [42]:
from pprint import pprint

def get_context(question, top_k=3):

    xq = retriever.encode([question]).tolist()


    results = index.query(
        vector=xq[0],
        top_k=top_k,
        include_metadata=True
    )

    contexts = [match['metadata']['context'] for match in results['matches']]

    return contexts

def extract_answer(question, context):
    results = []
    for c in context:
        answer = reader(question=question, context=c)
        answer["context"] = c
        results.append(answer)
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [44]:
question = "How to learn German?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'West Germanic',
  'context': 'The native language of Germans is German, a West Germanic '
             'language, related to and classified alongside English and Dutch, '
             'and sharing many similarities with the North Germanic and '
             'Scandinavian languages. Spoken by approximately 100 million '
             "native speakers, German is one of the world's major languages "
             'and the most widely spoken first language in the European Union. '
             'German has been replaced by English as the dominant language of '
             'science-related Nobel Prize laureates during the second half of '
             'the 20th century. It was a lingua franca in the Holy Roman '
             'Empire.',
  'end': 57,
  'score': 0.0002233815030194819,
  'start': 44},
 {'answer': 'Gesamtschule',
  'context': 'Germany has a comprehensive school known as the Gesamtschule. '
             'While some German schools such as the Gymnasium and the '
       

First of all after deleting the index i had to create a new one and re-write the whole code.
If you delete a `Pinecone/FAISS` index (or any vector store), all previously stored embeddings are lost! You must create a new index and **re-ingest/re-embed your data**, otherwise the retrieval function (`get_context`) has nothing to search.

The question `"How to learn German?"` had initially very low score which is valid since it didnt reply back to my question.

The other 2 responses are having quite small score as well (10⁻⁷ and 10⁻¹¹):
```
{'answer': 'Gesamtschule', 'score': 5.29e-07}  
{'answer': 'Fichte and Hegel', 'score': 1.85e-11}
```
which is expected too because the answer does not fit the question at all.